In [45]:
# pip install google-generativeai
import os
import gradio as gr
from dotenv import load_dotenv
import google.generativeai as palm
import webbrowser
import re

In [46]:
# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_PALM_API_KEY")
# Configure the API
palm.configure(api_key= GOOGLE_API_KEY)

In [47]:
def generate_text(prompt):
    completion = palm.generate_text(
        model="models/text-bison-001",
        prompt=prompt,
        temperature=0.7,
        max_output_tokens=800,
    )
    return completion.result

In [48]:
def parse_response(response):
    # Use regular expressions to find the sections more flexibly
    caviar_selection_match = re.search(r'Caviar Selection:\s*(.*)', response)
    champagne_pairing_match = re.search(r'Champagne Pairing:\s*(.*)', response)
    exclusive_insights_match = re.search(r'Exclusive Insights:\s*([\s\S]*)', response)

    caviar_selection = caviar_selection_match.group(1) if caviar_selection_match else None
    champagne_pairing = champagne_pairing_match.group(1) if champagne_pairing_match else None

    exclusive_insights = []
    if exclusive_insights_match:
        insights_section = exclusive_insights_match.group(1).strip()
        exclusive_insights = [line.strip().replace('*', '').strip() for line in insights_section.splitlines() if line.strip().startswith('*')]

    return caviar_selection, champagne_pairing, exclusive_insights

def caviar_coach(taste_profile, texture_preference, budget):
    image_url = "https://s.yimg.com/ny/api/res/1.2/.11X0wngxtZy7WtNp0r.2w--/YXBwaWQ9aGlnaGxhbmRlcjt3PTEyMDA7aD02NzQ-/https://media.zenfs.com/en/tasting_table_543/13b649b5330e192063fa2d41441150f3"
    prompt = f"""
    As a luxury caviar expert, provide a personalized recommendation based on the following preferences:
    Taste Profile: {taste_profile}
    Texture Preference: {texture_preference}
    Budget: ${budget}

    Please provide:
    1. A specific caviar recommendation
    2. A champagne pairing suggestion
    3. Some exclusive insights or tips for enjoying this caviar experience

    Format your response as follows:
    Caviar Selection: [Your recommendation]
    Champagne Pairing: [Your pairing suggestion]
    Exclusive Insights: [Your insights or tips]
    """

    try:
        # Get the generated response from the model
        response = generate_text(prompt)
        
        # Parse the response into sections using regular expressions
        caviar_selection, champagne_pairing, exclusive_insights = parse_response(response)
        
        # Format the response properly
        formatted_response = f"<div style='font-family: Arial, sans-serif; line-height: 1.6;'>"
        formatted_response += "<strong>Your Imperial Highness,</strong><br><br>"
        formatted_response += "Based on your exquisite preferences, I am honored to present our recommendation:<br><br>"
        caviar_selection.replace("**","")
        
        if caviar_selection:
            formatted_response += f"Caviar Selection: {caviar_selection}<br>"
        else:
            formatted_response += "<strong>Caviar Selection:</strong> Not available<br>"
        
        if champagne_pairing:
            formatted_response += f"Champagne Pairing: {champagne_pairing}<br><br>"
        else:
            formatted_response += "<strong>Champagne Pairing:</strong> Not available<br><br>"
        
        if exclusive_insights:
            formatted_response += "<strong>Exclusive Insights:</strong><br>"
            insights_html = "<br>".join([f"• {insight}" for insight in exclusive_insights if insight])  # Remove empty bullet points
            formatted_response += f"{insights_html}<br><br>"
        else:
            formatted_response += "<strong>Exclusive Insights:</strong> Not available<br><br>"
        # Add image to the response
        formatted_response += f"<img src='{image_url}' alt='Caviar Image' style='max-width: 50%; height: auto;'><br><br>"
        formatted_response += "May this experience elevate your senses and leave an indelible mark on your palate.<br><br>"
        formatted_response += "Bon appétit, Your Highness.<br><br>"
        formatted_response += "For more exclusive insights, visit us at: <a href='https://www.caviart.club' target='_blank'>here</a></div>"
        youtube_video_html = """<iframe width="560" height="315" src="https://www.youtube.com/embed/0QrUq7XWfvw" 
                             title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; 
                             encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>"""
        return formatted_response,youtube_video_html
    except Exception as e:
        return f"An error occurred: {str(e)}. Please check if the API key is set correctly."


In [49]:
iface = gr.Interface(
    fn=caviar_coach,
    inputs=[
        gr.Dropdown(["Mild & Delicate", "Rich & Buttery", "Bold & Intense"], label="Preferred Taste Profile"),
        gr.Dropdown(["Soft & Creamy", "Firm & Distinct"], label="Preferred Texture"),
        gr.Slider(minimum=50, maximum=1000, step=50, label="Budget (USD)")
    ],
    outputs=[
        gr.HTML(label="Your Royal Recommendation"),
        # gr.Video(label="Video Player")
        gr.HTML(label="YouTube Video Player")
    ],
    title="🍾 Caviart Club 🥂",
    description="Your Imperial Highness, please share your preferences for an exquisite caviar experience.",
)


In [50]:
if __name__ == "__main__":
    iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7866
Running on public URL: https://6cdad4a8d75cc09755.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
